周内效应——横向分析篇

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py
import numpy as np

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
# baselineasset = 'jrj.510310'
# baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [ ]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


大家好，我是格子衫小C，我们今天继续研究周内效应。

上次主要针对的是沪深300指数，我们发现了较明显的周内效应，今天我们分析一些别的资产。

纳斯达克指数

In [ ]:
asset = 'jqdata.513100_XSHG|1d'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='纳斯达克指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

这可以咱们大A股好多了，7年多的时间差不多5倍。

直接看看持有一天的情况。

In [ ]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1)

wrmt[0][['title', 'total']]

直接看胜率好了，因为纳斯达克指数整体是上涨的，所以胜率几乎都高于50%，但并不太明显（沪深300都有2天接近60%了）。

所以直接看PNL也没啥机会。

标准普尔指数

In [ ]:
asset = 'jqdata.513500_XSHG|1d'
asset = 'jrj.096001'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='标准普尔指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

标普没有纳斯达克指数那么好，而且2020年疫情过后涨幅不如纳指，但大体上也算是一个持续上涨趋势了。

In [ ]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1)

wrmt[0][['title', 'total']]

胜率上看来，最大也就稍大于55%。

恒生指数

In [ ]:
asset = 'jqdata.513600_XSHG|1d'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='恒生指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

恒生指数的数据没有那么长，只找到2015年以后的，这个看起来也不算太好，而且大的走势其实和沪深300很像。

2016、2017上涨，2018下跌，2019年前几个月把19年整年的涨完了。

这个走势大体相似的指数，如果周内效应不明显，也算是可以从另外一个角度验证周内效应。

In [ ]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1)

wrmt[0][['title', 'total']]

可以看到也都还是在50%上下波动，看起来至少是不明显。

这样看来，咱大A股的周内效应确实是特有的，美股和港股都没有明显的周内效应，特别是港股，大的走势和大A股很像，也不明显。

还是回头看看国内

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000016_XSHG|1d', 
                'jqdata.000300_XSHG|1d', 
                'jqdata.000905_XSHG|1d', 
                'jqdata.000032_XSHG|1d', 
                'jqdata.000033_XSHG|1d', 
                'jqdata.000034_XSHG|1d', 
                'jqdata.000036_XSHG|1d', 
                'jqdata.000037_XSHG|1d', 
                'jqdata.000038_XSHG|1d', 
                'jqdata.000039_XSHG|1d', 
                'jqdata.000040_XSHG|1d', 
                'jqdata.000043_XSHG|1d', 
                'jqdata.000044_XSHG|1d', 
                'jqdata.000045_XSHG|1d', 
                'jqdata.000046_XSHG|1d']
lstassetname = ['上证50', 
                '沪深300', 
                '中证500', 
                '上证能源', 
                '上证材料', 
                '上证工业', 
                '上证消费', 
                '上证医药', 
                '上证金融', 
                '上证信息', 
                '上证电信', 
                '超大盘', 
                '上证中盘', 
                '上证小盘', 
                '上证中小']

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='buyandhold',
    )

    paramsbuy = trdb2py.trading2_pb2.BuyParams(
        perHandMoney=1,
    )

    paramsinit = trdb2py.trading2_pb2.InitParams(
        money=10000,
    )

    s0.buy.extend([buy0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsInit.CopyFrom(paramsinit)        
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )
    
    lstparams.append(p0)

lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1, toImg=isStaticImg, width=width, height=height)

上面是我选的一些指数，可以看到上证消费几乎算是一直处于上升通道的指数，而上证能源，几乎是一个下降趋势的指数。

In [ ]:
lstassetcode = ['jqdata.513100_XSHG|1d',
                'jrj.096001',
                'jqdata.513600_XSHG|1d',
                'jqdata.000016_XSHG|1d', 
                'jqdata.000300_XSHG|1d', 
                'jqdata.000905_XSHG|1d', 
                'jqdata.000032_XSHG|1d', 
                'jqdata.000033_XSHG|1d', 
                'jqdata.000034_XSHG|1d', 
                'jqdata.000036_XSHG|1d', 
                'jqdata.000037_XSHG|1d', 
                'jqdata.000038_XSHG|1d', 
                'jqdata.000039_XSHG|1d', 
                'jqdata.000040_XSHG|1d', 
                'jqdata.000043_XSHG|1d', 
                'jqdata.000044_XSHG|1d', 
                'jqdata.000045_XSHG|1d', 
                'jqdata.000046_XSHG|1d']
lstassetname = ['纳斯达克指数',
                '标准普尔指数',
                '恒生指数',
                '上证50', 
                '沪深300', 
                '中证500', 
                '上证能源', 
                '上证材料', 
                '上证工业', 
                '上证消费', 
                '上证医药', 
                '上证金融', 
                '上证信息', 
                '上证电信', 
                '超大盘', 
                '上证中盘', 
                '上证小盘', 
                '上证中小']

fv0 = {'title': [], 'week1': [], 'week2': [], 'week3': [], 'week4': [], 'week5': [], 'mean': [], 'var': [], 'std': []}

for ai in range(0, len(lstassetcode)):
    fv0['title'].append(lstassetname[ai])
    
    for i in range(0, 5):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday2',
            vals=[lststart[i], calcweekday2val2(i + 1, 1)],
        )

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[trdb2py.nextWeekDay(lststart[i], 1)],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(lstassetcode[ai]),
        )

        s0.buy.extend([buy0])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        p0 = trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(lstassetcode[ai])],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='{}买入{}持有{}天'.format(lsttitle[i], lstassetname[ai], 1),
        )
        
        curpnl = trdb2py.simTrading(trdb2cfg, p0)
        ar = trdb2py.analysisResult(curpnl)
        fv0['week{}'.format(i + 1)].append(ar['winRate'])
        
    fv0['mean'].append(np.mean([fv0['week5'][ai], fv0['week1'][ai], fv0['week2'][ai], fv0['week3'][ai], fv0['week4'][ai]]))
    fv0['var'].append(np.var([fv0['week5'][ai], fv0['week1'][ai], fv0['week2'][ai], fv0['week3'][ai], fv0['week4'][ai]]))
    fv0['std'].append(np.std([fv0['week5'][ai], fv0['week1'][ai], fv0['week2'][ai], fv0['week3'][ai], fv0['week4'][ai]]))    

pdar = pd.DataFrame(fv0)
pdar

上面列了一共18种指数的统计（90种情况），除了前面纳指、标普、恒指外，后面全部都是国内指数。

从标准差来看，国内指数比国外的指数高接近一倍。

然后就是中盘要明显过小盘，小盘明显过超大盘。
同理，沪深300要比中证500明显，中证500要比上证50明显。

从行业来看，工业的突出一点，但工业主要是由于周四跌的概率过大。

小盘股，周二涨的概率稍好于周一。

大部分的国内指数，只考虑胜率，周四买入，持有3天，大概率是赚的。
我估计，前面跑出来的策略参数，在这些指数里面，有一定的通用性，所以就直接拿前面的29日均线上，周四买入持有4天，29均线下，周一买入持有4天

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000016_XSHG|1d', 
                'jqdata.000300_XSHG|1d', 
                'jqdata.000905_XSHG|1d', 
                'jqdata.000032_XSHG|1d', 
                'jqdata.000033_XSHG|1d', 
                'jqdata.000034_XSHG|1d', 
                'jqdata.000036_XSHG|1d', 
                'jqdata.000037_XSHG|1d', 
                'jqdata.000038_XSHG|1d', 
                'jqdata.000039_XSHG|1d', 
                'jqdata.000040_XSHG|1d', 
                'jqdata.000043_XSHG|1d', 
                'jqdata.000044_XSHG|1d', 
                'jqdata.000045_XSHG|1d', 
                'jqdata.000046_XSHG|1d']
lstassetname = ['上证50', 
                '沪深300', 
                '中证500', 
                '上证能源', 
                '上证材料', 
                '上证工业', 
                '上证消费', 
                '上证医药', 
                '上证金融', 
                '上证信息', 
                '上证电信', 
                '超大盘', 
                '上证中盘', 
                '上证小盘', 
                '上证中小']

# tsStart = 0
# tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(29)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.29'],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

    paramsbuy = trdb2py.trading2_pb2.BuyParams(
        perHandMoney=1,
    )

    paramsinit = trdb2py.trading2_pb2.InitParams(
        money=10000,
    )

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1, toImg=isStaticImg, width=width, height=height)

可以看到，在时间不变的情况下，就算是最差的能源，也有122%的回报率了，而最好的消费和信息，回报都在500%多。

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000016_XSHG|1d', 
                'jqdata.000300_XSHG|1d', 
                'jqdata.000905_XSHG|1d', 
                'jqdata.000032_XSHG|1d', 
                'jqdata.000033_XSHG|1d', 
                'jqdata.000034_XSHG|1d', 
                'jqdata.000036_XSHG|1d', 
                'jqdata.000037_XSHG|1d', 
                'jqdata.000038_XSHG|1d', 
                'jqdata.000039_XSHG|1d', 
                'jqdata.000040_XSHG|1d', 
                'jqdata.000043_XSHG|1d', 
                'jqdata.000044_XSHG|1d', 
                'jqdata.000045_XSHG|1d', 
                'jqdata.000046_XSHG|1d']
lstassetname = ['上证50', 
                '沪深300', 
                '中证500', 
                '上证能源', 
                '上证材料', 
                '上证工业', 
                '上证消费', 
                '上证医药', 
                '上证金融', 
                '上证信息', 
                '上证电信', 
                '超大盘', 
                '上证中盘', 
                '上证小盘', 
                '上证中小']

tsStart = 0
tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(29)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.29'],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

    paramsbuy = trdb2py.trading2_pb2.BuyParams(
        perHandMoney=1,
    )

    paramsinit = trdb2py.trading2_pb2.InitParams(
        money=10000,
    )

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1, toImg=isStaticImg, width=width, height=height)

数据放到2005年，就是这样的。

最低的能源也有接近500%的回报了，最高接近40倍。

按道理，我们根据不同指数，应该是能跑出不同的参数，最终情况应该会比这个更好，前面看胜率也能看到，虽然基本上都是周四跌的概率最大，但其它日子还是有差别的